In [1]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to avoid too many logging messages
import pandas as pd, numpy as np, random, shutil
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
import sklearn
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow_probability as tfp
import wandb
import yaml

#from IPython import display as ipd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

/Users/makoto/homebrew/Caskroom/miniforge/base/envs/rsna2023/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: m-noda. Use `wandb login --relogin` to force relogin


True

### Configs

In [3]:
class CFG:
    wandb         = True

    '''
    competition   = 'rsna-atd' 
    _wandb_kernel = 'm-noda'
    debug         = False
    comment       = 'EfficientNetV1B1-512x512-lr2-vflip'
    exp_name      = 'baseline-v5: ds-v3 + multi_head' 
    '''
    
    # use verbose=0 for silent, vebose=1 for interactive,
    verbose      = 0
    display_plot = True

    # device
    #device = "TPU-VM" #or "GPU"

    model_name = 'EfficientNetV1B1'

    seed = 123

    folds = 5
    
    # which folds to train
    selected_folds = [0, 1]

    # size of the image
    img_size = [512, 512]

    batch_size = 8
    epochs = 12

    loss      = 'BCE & CCE'
    
    # optimizer
    optimizer = 'Adam'

    # augmentation
    augment   = True

    # scale-shift-rotate-shear
    transform = 0.90  # transform prob
    fill_mode = 'constant'
    rot    = 2.0
    shr    = 2.0
    hzoom  = 50.0
    wzoom  = 50.0
    hshift = 10.0
    wshift = 10.0

    # flip
    hflip = True
    vflip = True

    # clip
    clip = False

    # lr-scheduler
    scheduler   = 'exp' # cosine

    # dropout
    drop_prob   = 0.6
    drop_cnt    = 5
    drop_size   = 0.05
    
    # cut-mix-up
    mixup_prob = 0.0
    mixup_alpha = 0.5
    
    cutmix_prob = 0.0
    cutmix_alpha = 2.5

    # pixel-augment
    pixel_aug = 0.90  # prob of pixel_aug
    sat  = [0.7, 1.3]
    cont = [0.8, 1.2]
    bri  = 0.15
    hue  = 0.05

    # test-time augs
    tta = 1
    
    # target column
    target_col  = [ "bowel_injury", "extravasation_injury", "kidney_healthy", "kidney_low",
                   "kidney_high", "liver_healthy", "liver_low", "liver_high",
                   "spleen_healthy", "spleen_low", "spleen_high"] # not using "bowel_healthy" & "extravasation_healthy"


### Seeding for reproducibility

In [4]:
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    tf.random.set_seed(SEED)
    print('seeding done!!!')
seeding(CFG.seed)

seeding done!!!


### Load train data, test data

In [23]:

# make dataframes
train_df = pd.read_csv("/rsna2023atd_files/train.csv")
img_level_df = pd.read_csv("/rsna2023atd_files/image_level_labels.csv")
train_df = train_df.merge(img_level_df, on=['patient_id'], how='right')
train_df['image_path'] = (f'/train_png' + '/' + train_df.patient_id.astype(str) + '_' + train_df.series_id.astype(str) + '_' + train_df.instance_number.astype(str) + '.png')
train_df = train_df.drop_duplicates()

test_paths = glob('./test_png/*_*_*.png')
test_df = pd.DataFrame(test_paths, columns=["image_path"])
test_df['patient_id'] = test_df.image_path.map(lambda x: x.split('_')[-3].replace('png/','')).astype(int)
test_df['series_id'] = test_df.image_path.map(lambda x: x.split('_')[-2]).astype(int)
test_df['instance_number'] = test_df.image_path.map(lambda x: x.split('_')[-1].replace('.png','')).astype(int)

In [38]:
test_df.head()

,image_path,patient_id,series_id,instance_number
0,./test_png/48843_62825_0.png,48843,62825,0
1,./test_png/63706_39279_0.png,63706,39279,0
2,./test_png/50046_24574_0.png,50046,24574,0


In [24]:
train_df.head()

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury,series_id,instance_number,injury_name,image_path
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,362,Active_Extravasation,./train_png/10004_21057_362.png
1,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,363,Active_Extravasation,./train_png/10004_21057_363.png
2,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,364,Active_Extravasation,./train_png/10004_21057_364.png
3,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,365,Active_Extravasation,./train_png/10004_21057_365.png
4,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1,21057,366,Active_Extravasation,./train_png/10004_21057_366.png


In [40]:
tf.io.gfile.exists(train_df.image_path.iloc[0]), tf.io.gfile.exists(test_df.image_path.iloc[0])

(False, True)